# Example of using Llamaindex framework for Retrieval Augmented Generation
This notebook shows how to run Llamaindex framework locally to create virtual AI assistant based on RAG (Retrieval Augmented Generation).
For dataset to search for source information wikipedia articles about cryptocurrencies were used.

## Llamaindex setup
### Download llamafile
Download the llamafile with model. Llamafile can contain any LLM. Framework enabled to run it as a local server and use via API. 
TinyLlama-1.1B-Chat-v1.0 model is used for purpose of this example

`wget https://huggingface.co/jartine/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile`

Make executable 

`chmod +x TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile`

Run in server mode

`./TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile --server --nobrowser --embedding --port 8080`

### Install Llamaindex Python library

In [16]:
# Install llama-index
!pip install llama-index 
# Install llamafile integrations and SimpleWebPageReader
!pip install llama-index-embeddings-llamafile llama-index-llms-llamafile llama-index-readers-web

## Configuration

In [17]:
# Configure LlamaIndex
from llama_index.core import Settings
from llama_index.embeddings.llamafile import LlamafileEmbedding
from llama_index.llms.llamafile import Llamafile
from llama_index.core.node_parser import SentenceSplitter

#configure object to encode text into vector using started endpoint
Settings.embed_model = LlamafileEmbedding(base_url="http://localhost:8080")

#configure object that will use model endpoint
Settings.llm = Llamafile(
    base_url="http://localhost:8080",
    temperature=0,
    seed=0
)

#configure split text to chunks 
Settings.transformations = [
    SentenceSplitter(
        chunk_size=256, 
        chunk_overlap=5
    )
]

In [18]:
# Load local data with some cryptocurrencies descriptions
from llama_index.core import SimpleDirectoryReader
# create reader for local documents
local_doc_reader = SimpleDirectoryReader(input_dir='./data/cryptocurrency_wikipedia')
# create collection that will contain all documents used for retrieval
docs = local_doc_reader.load_data(show_progress=True)


Loading files: 100%|██████████| 3/3 [00:00<00:00, 2713.59file/s]


In [19]:
# Add Wikipedia pages
from llama_index.readers.web import SimpleWebPageReader
urls = [
    'https://en.wikipedia.org/wiki/Bitcoin',
    'https://en.wikipedia.org/wiki/Ethereum',
    'https://en.wikipedia.org/wiki/Dogecoin'
]
# create reader that can fetch websites content
web_reader = SimpleWebPageReader(html_to_text=True)
# add fetched content to docs collection
docs.extend(web_reader.load_data(urls))

In [20]:
from llama_index.core import VectorStoreIndex

# create index storage with embedded documents
index = VectorStoreIndex.from_documents(
    docs,
    show_progress=True,
)

# dump storage locally
index.storage_context.persist(persist_dir="./storage")

Parsing nodes:   0%|          | 0/6 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/545 [00:00<?, ?it/s]

In [22]:
# create query type engine to ask questions to llm providing documents from index
query_engine = index.as_query_engine()
# ask questions about data from storage
print(query_engine.query("What is Coinye?"))

Coinye is a cryptocurrency that is based on the Ethereum blockchain. It is a decentralized platform that allows users to create and trade digital assets without the need for intermediaries or centralized authorities. Coinye's main feature is its decentralized and secure blockchain technology, which allows for faster and more efficient transactions compared to traditional financial systems. Additionally, Coinye offers a range of services, including trading, investment, and lending, making it a versatile and useful tool for cryptocurrency enthusiasts and investors.</s>


In [26]:
print(query_engine.query("Is Dogecoin stable?"))

Yes, Dogecoin is considered stable and has a high degree of consensus. It is a decentralized cryptocurrency that features the face of Kabosu, a popular meme character, as its logo and namesake. The coin's development community has a strong online presence, and it has reached a peak market capitalization of over US$85 billion as of May 2021. The blockchain's attestation system ensures that the blockchain is stable and that validators are rewarded for making valid proposals and attestations. Dogecoin's attestations are given a weight equal to its stake or 32, whichever is less, and it has a high degree of consensus due to its decentralized nature.</s>


In [28]:
print(query_engine.query("Is Bitcoin good for environment?"))

Yes, Bitcoin is considered to be a good option for environment as it does not require any physical infrastructure, such as mining facilities, to operate. It is a decentralized and secure cryptocurrency that uses blockchain technology to verify transactions and maintain a decentralized ledger. This means that Bitcoin does not require any intermediaries or third-party entities to process transactions, which reduces the carbon footprint associated with traditional financial systems. Additionally, Bitcoin's energy consumption is significantly lower than that of traditional financial systems, making it a more sustainable option for the environment.</s>
